In [1]:
import meshcat
import meshcat.geometry as g
from jaxlie import SE3, SO3
import jax.numpy as jnp
import numpy as np
import jax
from jax import Array

from sdf_world.sdf_world import *
from sdf_world.robots import *

In [2]:
world = SDFWorld()
world.show_in_jupyter()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [3]:
ground = Box(world.vis, "ground", lengths=[2, 2, 0.1], color="gray")
ground.set_translate([0,0,-ground.lengths[-1]/2])

plate_config = dict(lengths=[0.4, 0.4, 0.05], color="darkgray")
start = Box(world.vis, "start", **plate_config)
goal = Box(world.vis, "goal", **plate_config)
start.set_translate([0.5,-0.3,start.lengths[-1]/2])
goal.set_translate([0.5,0.3,goal.lengths[-1]/2])

obstacle = Sphere(world.vis, "obstacle", 0.2, "red", 0.3)
obstacle.set_translate([0.5, 0., 0.5])

In [4]:
panda_model = RobotModel(PANDA_URDF, PANDA_PACKAGE)
panda = Robot(world.vis, "panda", panda_model)
panda.reduce_dim([7,8], [0.04, 0.04])

concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts


In [5]:
obstacles = [ground, obstacle]
env = SDFContainer(obstacles, 0.01)

### test motion planning

In [6]:
q_start = jnp.array([-0.7,0.2,0,-1.57,0,1.8,0])
q_goal = jnp.array([0.7,0.2,0,-1.57,0,1.8,0])
panda.set_joint_angles(q_start)

In [25]:
from sdf_world.nlp import *
prob = NLP()
prob.add_var("q0", 7, panda.lb, panda.ub)
prob.add_var("q1", 7, panda.lb, panda.ub)
prob.add_var("q2", 7, panda.lb, panda.ub)
prob.add_var("q3", 7, panda.lb, panda.ub)
prob.add_var("q4", 7, panda.lb, panda.ub)

In [48]:
prob.add_con("init", 7, ["q0"], lambda q0:q0, eq=q_start, est_sparsity=True)
prob.add_con("goal", 7, ["q4"], lambda q4:q4, eq=q_goal, est_sparsity=True)
@jax.jit
def penetration(**kwargs):
    qs = jnp.vstack(kwargs.values())
    points = jnp.vstack(jax.vmap(panda.get_surface_points_fn)(qs))
    return env.penetration_sum(points)
prob.add_con("col", 1, ["q1","q2","q3"], penetration, eq=0.)

def min_dist_cost(q0, q1, q2, q3, q4):
    qs = jnp.vstack([q0, q1, q2, q3, q4])
    diff = qs[1:] - qs[:-1]
    return jnp.sum(diff**2)
prob.add_objective(min_dist_cost)

In [27]:
prob.build()

In [64]:
x0 = jnp.linspace(q_start, q_goal, 5).flatten()
x, info = prob.solve(x0, tol=0.5)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       35
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       35
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       35
                     variables with only upper bounds:        0
Total number of equality constraints.................:       15
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.8999995e-01 6.49e+00 1.19e-07   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [71]:
qs = x.reshape(-1, 7)
i = 0

In [74]:
panda.set_joint_angles(qs[i])
i += 1

In [109]:
points = panda.get_surface_points_fn(qs[-1])
penets = jax.vmap(env.sdfs[1].penetration, in_axes=(0,None))(points, 0.01)

In [110]:
penets[penets > 0]

Array([], dtype=float32)

In [112]:
panda_model.links["panda_link0"] == panda_model.root_link

True

In [98]:
penetration(qs[-1])

Array(0.0381052, dtype=float32)

In [97]:
panda.set_joint_angles(qs[-1])

In [ ]:
panda1 = Robot(world.vis, "panda1", panda_model)
panda1.reduce_dim([7,8], [0.04, 0.04])

In [ ]:
panda1.reduce_dim([7,8], [0.04, 0.04])

In [ ]:
panda1.set_pose(SE3.from_translation(jnp.array([0, -0.5,0])))

In [ ]:

panda2 = Robot(world.vis, "panda2", panda_model)
panda2.reduce_dim([7,8], [0.04, 0.04])
panda2.set_pose(SE3.from_translation(jnp.array([0, 0.5,0])))

In [ ]:
box = Box(world.vis, "box1", [0.5, 0.5, 0.2], [0.2,0.2,0.2], 0.8)

In [ ]:
for i in range(100):
    panda1.set_joint_angles(panda1.get_random_config())
    panda2.set_joint_angles(panda2.get_random_config())
    if panda1.q[-1] != 0.04:
        break

In [ ]:
q = panda1.get_random_config()

In [ ]:
if len(panda1.fix_idx) != 0:
    qnew = np.zeros(panda1.model.dof)
    np.put(qnew, panda1.free_idx, q)
    np.put(qnew, panda1.fix_idx, panda1.fix_value)
    self.q = qnew
else:
    self.q = np.asarray(q)

In [ ]:
qnew = np.empty(panda_model.dof)

In [ ]:
np.put(qnew, panda1.fix_idx, panda1.fix_val)

In [ ]:
world.show_in_jupyter()

In [ ]:
from matplotlib.colors import rgb2hex

In [ ]:
to_numpy = lambda x: np.array(x, dtype=np.float64)
def mat_from_translate(translate):
    translate = jnp.array(translate)
    return to_numpy(SE3.from_translation(translate).as_matrix())
def to_numpy_tfmat(x):
    if isinstance(x, SE3):
        return to_numpy(x.as_matrix())
    elif isinstance(x, Array):
        if x.shape == (7,):
            return to_numpy(SE3(x).as_matrix())
        else:
            return to_numpy(SE3.from_matrix(x))
    elif isinstance(x, np.ndarray):
        return x
    raise NotImplementedError(type(x))
def to_SE3(x):
    if isinstance(x, np.ndarray):
        return SE3.from_matrix(x)
    raise NotImplementedError(type(x))

class MeshCatObject:
    def __init__(self, vis, name):
        self.vis = vis
        self.name = name
        self.handle = self.vis[name]
        self.handle.delete()
        self.pose = SE3.identity()
    
    def load(self):
        raise NotImplementedError()
    
    def __del__(self):
        self.handle.delete()

    def set_pose(self, pose:SE3):
        self.pose = pose
        self.handle.set_transform(to_numpy_tfmat(pose))

class Box(MeshCatObject):
    def __init__(self, vis, name, lengths, color, alpha=1.):
        assert len(lengths) == 3
        super().__init__(vis=vis, name=name)
        if not isinstance(color, int):
            color = int(rgb2hex(color).replace("#", "0x"), 16)
        self.lengths = lengths
        self.color = color
        self.alpha = alpha
        self.load()

    def load(self):
        obj = g.Box(self.lengths)
        material = g.MeshLambertMaterial(color=self.color, alpha=self.alpha)
        self.handle.set_object(obj, material)

class Mesh(MeshCatObject):
    def __init__(self, vis, name, path):
        super().__init__(vis=vis, name=name)
        self.path = path
        self.mesh = trimesh.load(path)
        if isinstance(self.mesh, trimesh.Scene):
            self.mesh = self.mesh.dump(True)
        self.load()

    def load(self):
        exp_obj = export_obj(self.mesh)
        obj = g.ObjMeshGeometry.from_stream(
            trimesh.util.wrap_as_stream(exp_obj))
        self.vis[self.name].set_object(obj)


class Frame(MeshCatObject):
    def __init__(self, vis, name, length=0.1, r=0.02):
        super().__init__(vis=vis, name=name)
        self.length = length
        self.r = r
        self.load()
    
    def load(self):
        axes = ["x", "y", "z"]
        colors = [0xff0000, 0x00ff00, 0x0000ff]
        length_mat = np.full((3,3), self.r)
        np.fill_diagonal(length_mat, self.length)
        tf_mat = np.diag([self.length/2]*3)
        for i in range(3):
            color = colors[i]
            obj = g.Box(length_mat[i])
            material = g.MeshLambertMaterial(color=color)
            self.handle[axes[i]].set_object(obj, material)
            self.handle[axes[i]].set_transform(mat_from_translate(tf_mat[i]))

In [ ]:
from pathlib import Path
from pybullet_suite import PANDA_URDF
urdf = PANDA_URDF
package = Path(PANDA_URDF).parent

In [ ]:
__vsc_ipynb_file__

In [ ]:
import xml.etree.ElementTree as ET
from typing import *

def str2arr(string):
    if string is None:
        return None
    return np.array(string.split(" ")).astype(float)

class Link:
    def __init__(self, name, xyz=None, rpy=None, visual_mesh=None, collision_mesh=None, package=None):
        self.name = name
        self.xyz = str2arr(xyz)
        self.rpy = str2arr(rpy)
        self.parent: Joint = None
        self.child: Joint = None
        self.visual_mesh_path = visual_mesh
        self.collision_mesh_path = collision_mesh
        
        if self.has_mesh:
            self.visual_mesh_path = self.get_mesh_path(visual_mesh, package)
            self.collision_mesh_path = self.get_mesh_path(collision_mesh, package)
            self.visual_mesh = self.load_mesh(self.visual_mesh_path)
            self.collision_mesh = self.load_mesh(self.collision_mesh_path)

    @property
    def has_mesh(self):
        return (self.visual_mesh_path is not None) or (self.collision_mesh_path is not None)
    
    @property
    def has_mesh_offset(self):
        return (self.xyz is not None) or (self.rpy is not None)

    def load_mesh(self, path):
        mesh = trimesh.load(path)
        if isinstance(mesh, trimesh.Scene):
            # print(f"merging mesh scene: {path}")
            mesh = mesh.dump(True)
        return mesh
    
    def get_meshcat_obj(self, type="visual"):
        if type == "visual":
            mesh = self.visual_mesh
        else:
            raise NotImplementedError("visual mesh only")
        exp_obj = export_obj(mesh)
        return g.ObjMeshGeometry.from_stream(
            trimesh.util.wrap_as_stream(exp_obj))
    
    def get_T_link_offset(self):
        xyz = jnp.array(self.xyz)
        rot = SO3.from_rpy_radians(*self.rpy)
        return SE3.from_rotation_and_translation(rot, xyz)
    
    def get_mesh_path(self, path:str, package:Union[str,Path]):
        if isinstance(package, str):
            return path.replace("package:/", package)
        elif isinstance(package, Path):
            return path.replace("package:/", package.as_posix())
    
    def __repr__(self):
        return f"Link:{self.name}"

class Joint:
    def __init__(self, name, joint_type, xyz, rpy, parent, child, 
                 axis=None, lb=None, ub=None):
        self.name = name
        self.joint_type = joint_type
        self.xyz = str2arr(xyz)
        self.rpy = str2arr(rpy)
        self.parent: Link = parent
        self.child: Link = child
        self.axis = str2arr(axis)
        self.lb = None if lb is None else float(lb)
        self.ub = None if lb is None else float(ub)
    
    @property
    def actuated(self):
        return self.joint_type != "fixed"
    
    def get_T_offset(self):
        xyz = jnp.array(self.xyz)
        rot = SO3.from_rpy_radians(*self.rpy)
        return SE3.from_rotation_and_translation(rot, xyz)

    def get_T_joint(self, q):
        if self.joint_type == "revolute":
            #assume that revolute joint axis is 0 0 1
            return SE3.from_rotation(SO3.from_rpy_radians(0,0,q))
        elif self.joint_type == "prismatic":
            return SE3.from_translation(jnp.array(self.axis * q))

    def __repr__(self):
        return f"Joint:{self.name}"
    
class Robot(MeshCatObject):
    def __init__(self, vis, name, urdf, package, compile_fk=True, visualize=True):
        super().__init__(vis, name)
        self.urdf = urdf
        self.packge = package
        self.links: Dict[str,Link] = {}
        self.joints: Dict[str,Joint] = {}
        self.fk_fn = None #jitted FK
        self.fk = None #compiled FK
        self.dof = None
        self.view_col_mesh = False #not implemented
        self.view_visual_mesh = True
        self.q = None
        self.lb = None
        self.ub = None
        self.neutral = None

        self.parse_urdf()
        if compile_fk:
            self.compile_fk()
        if visualize:
            self.load()
            self.set_joint_angles(self.q)

    def load(self):
        for link in self.links.values():
            # for now, only supports visual mesh
            if link.has_mesh:
                obj = link.get_meshcat_obj("visual")
                self.handle["visual"][link.name].set_object(obj)

    def show(self):
        Ts = np.asarray(self.fk(self.q), dtype=np.float64)
        i = 0
        for i, link in enumerate(self.links.values()):
            if not link.has_mesh: continue
            self.handle["visual"][link.name].set_transform(Ts[i])
            i += 1


    def set_joint_angles(self, q:Array):
        assert len(q) == self.dof
        self.q = q
        self.show()
    
    def parse_urdf(self):
        tree = ET.parse(self.urdf)
        root = tree.getroot()

        #parse links
        for link in root.iter("link"):
            xyz = rpy = None
            name = link.attrib["name"]
            visual_mesh = collision_mesh = None
            if link.find("visual") is not None:
                visual_mesh = link.find("visual").find("geometry").find("mesh").attrib["filename"]
                if link.find("visual").find("origin") is not None:
                    xyz = link.find("visual").find("origin").attrib["xyz"]
                    rpy = link.find("visual").find("origin").attrib["rpy"]
            if link.find("collision") is not None:
                collision_mesh = link.find("collision").find("geometry").find("mesh").attrib["filename"]
            self.links[name] = Link(
                name, xyz, rpy, visual_mesh, collision_mesh, self.packge)
        last_link_name = name

        #parse joints
        self.joints = {}
        for joint in root.iter("joint"):
            name = joint.attrib["name"]
            joint_type = joint.attrib["type"]
            xyz = joint.find("origin").attrib["xyz"]
            rpy = joint.find("origin").attrib["rpy"]
            parent_name = joint.find("parent").attrib["link"]
            parent = self.links[parent_name]
            child_name = joint.find("child").attrib["link"]
            child = self.links[child_name]
            axis = lb = ub = None
            if joint_type != "fixed":
                axis = joint.find("axis").attrib["xyz"]
                lb = joint.find("limit").attrib["lower"]
                ub = joint.find("limit").attrib["upper"]
            self.joints[name] = Joint(name, joint_type, xyz, rpy, parent, child, axis, lb, ub)
            parent.child = self.joints[name]
            child.parent = self.joints[name]
        
        #set parameters
        self.root_link = self.get_root_link(self.links[last_link_name])
        self.dof = 0
        for joint in self.joints.values():
            if joint.joint_type != "fixed":
                self.dof += 1
        self.lb = np.array([joint.lb for joint in self.joints.values() if joint.actuated])
        self.ub = np.array([joint.ub for joint in self.joints.values() if joint.actuated])
        self.neutral = (self.lb + self.ub) / 2
        self.q = self.neutral
        
    def compile_fk(self):
        def fk(q):
            Tlink = {}
            Tlink[self.root_link.name] = SE3.identity()
            dof_idx = 0
            for link in self.links.values():
                if link.name in Tlink: continue
                parent_joint = link.parent
                parent_link = parent_joint.parent
                T = Tlink[parent_link.name] @ parent_joint.get_T_offset()
                if parent_joint.joint_type != "fixed":
                    theta = q[dof_idx]
                    T = T @ parent_joint.get_T_joint(theta)
                    dof_idx += 1
                if link.has_mesh_offset:
                    T = T @ link.get_T_link_offset()
                Tlink[link.name] = T
            return jnp.stack([T.as_matrix() for T in Tlink.values()], axis=0)
        self.fk_fn = jax.jit(fk)
        self.fk = self.fk_fn.lower(jnp.zeros(self.dof)).compile()

    def get_root_link(self, link:Link):
        while True:
            if link.parent is None:
                break
            link = link.parent
        return link
    
    def get_random_config(self):
        return np.random.uniform(self.lb, self.ub)
        

In [ ]:
frame = Frame(world.vis, "debug", length=0.3)

In [ ]:
robot = Robot(world.vis, "panda", urdf, package)

In [ ]:
%timeit robot.set_joint_angles(robot.neutral)

In [ ]:
%timeit robot.fk(robot.neutral)

In [ ]:
self.lb = np.array([joint.lb for joint in self.joints.values() if joint.actuated])
self.ub = np.array([joint.ub for joint in self.joints.values() if joint.actuated])
self.neutral = (self.lb + self.ub) / 2

In [ ]:
robot.set_joint_angles(jnp.zeros(9))

In [ ]:
roboturdf.load()

In [ ]:
robot.set_joint_angles(jnp.array([-0.,0.3,0.3,-0.2,0.3,0.3,0.2,0.04,0.04]))

In [ ]:
import trimesh

In [ ]:
mesh_path = roboturdf.links["panda_link6"].get_mesh_path(package, "visual")

In [ ]:
mesh = trimesh.load(mesh_path)

In [ ]:
world.vis.set_object(obj)

In [ ]:
mesh = Mesh(world.vis, "test", mesh_path)

In [ ]:
del mesh

In [ ]:
mesh.load()

In [ ]:
world.vis.delete()

In [ ]:
roboturdf.set_view(view_col_mesh=False, view_visual_mesh=True)

In [ ]:
q = jnp.array([0.05,0,0,0,0,0,0,0.,0.])
#roboturdf.fk(q)
roboturdf.set_joint_angles(q)

In [ ]:
link = roboturdf.links["panda_link0"]

In [ ]:
file = link.visual_mesh_path.replace("package:/", package.as_posix())
obj = g.ObjMeshGeometry.from_file(file)

In [ ]:
obj = roboturdf.get_mesh(link.visual_mesh_path)

In [ ]:
box = Box(world.vis, "box1", [0.1, 0.1, 0.1], [1,0,0], 0.8)

In [ ]:
del box

In [ ]:
panda = world.vis["panda"]
panda.delete()

In [ ]:
panda.set_object(obj)

In [ ]:
roboturdf.handle["visual"]["link0"].set_object(obj)

In [ ]:
link

In [ ]:
col = self.handle["collision"]
visual = self.handle["visual"]
for link in self.links.values():
    if link.visual_mesh is not None:
        visual[link.name].set_object(self.get_mesh(link.visual_mesh))
    if link.collision_mesh is not None:
        col[link.name].set_object(self.get_mesh(link.collision_mesh))

In [ ]:
roboturdf.load()

In [ ]:
ro

In [ ]:
roboturdf.packge

In [ ]:
roboturdf.fk

In [ ]:
roboturdf.compile_fk()

In [ ]:
%timeit roboturdf.fk(np.random.random(9))

In [ ]:
roboturdf.fk(np.random.random(9))

In [ ]:
self.links["panda_link0"].collision_mesh

In [ ]:

class Panda(MeshCatObject):
    def __init__(self, vis, name, urdf_path, package_path):
        super().__init__(vis=vis, name=name)
        self.urdf_path = urdf_path #urdf should be in path, meshes should be in path/meshes/{collision, visual}/{link}.obj or dae
        self.package_path = package_path
        self.links = ["link0", "link1", "link2", "link3", "link4", "link5", "link6", "link7", "hand", "finger", "finger"]
        self.paths = [f"./Panda/meshes/visual/{link}.dae" for link in self.links]
        self.links[-2] = "finger_l"
        self.links[-1] = "finger_r"
        self.panda = self.vis["panda"]
        
        self.link_dict, self.joint_dict = None, None
        self.load_urdf()

        self.fk_fn = jax.jit(self.get_fk_fn())
        self.show()

In [ ]:
frame = Frame(world.vis, "f1")

In [ ]:
frame.set_pose(mat_from_translate([1,0.1, 0.3]))

In [ ]:
del frame

In [ ]:
frame.handle.delete()

In [ ]:
frame.length

In [ ]:
frame = world.vis["frame"]

In [ ]:
r = 0.1
length = 1.

In [ ]:
x, y, z = np.eye(3)

In [ ]:
frame = world.vis['frame']
frame.delete()

In [ ]:
axes = ["x", "y", "z"]
colors = [0xff0000, 0x00ff00, 0x0000ff]
length_mat = np.full((3,3), r)
np.fill_diagonal(length_mat, length)
tf_mat = np.diag([length/2]*3)
for i in range(3):
    color = colors[i]
    obj = g.Box(length_mat[i])
    material = g.MeshLambertMaterial(color=color)
    frame[axes[i]].set_object(obj, material)
    frame[axes[i]].set_transform(mat_from_translate(tf_mat[i]))

In [ ]:
frame.delete()

In [ ]:
tf_mat

In [ ]:
length_mat

In [ ]:
frame.delete()

In [ ]:
obj = g.Box([length, r, r])
material = g.MeshLambertMaterial(color=0xff0000)
frame["x"].set_object(obj, material)

In [ ]:
tf = SE3.from_translation(jnp.array([length/2, 0, 0])).as_matrix()
tf = to_numpy(tf)
frame["x"].set_transform(tf)

In [ ]:
frame["x"]

In [ ]:
box = Box(world.vis, "box", [0.5, 0.4, 0.3], [1,0,0], 0.8)

In [ ]:
mat = to_numpy(SE3.from_translation(jnp.array([1,0,0])).as_matrix())
box.vis["box"].set_transform(mat)

In [ ]:
pose = SE3.from_translation(jnp.array([0, 0, 0.5]))

In [ ]:
box.set_pose(SE3.from_translation(jnp.array([0, 0, 0.3])))

In [ ]:
del box

In [ ]:
world.vis.delete()

In [ ]:
def create_box(name, lengths, color, alpha=1.):
    if not isinstance(color, int):
        color = int(rgb2hex(color).replace("#", "0x"), 16)
    obj = g.Box(lengths)
    material = g.MeshLambertMaterial(color=color, opacity=alpha)
    world.vis[name].set_object(obj, material)

def create_table(lengths, offsets, color, alpha, ):
    """if offset_z=0, ground-level"""
    create_box()
    

In [ ]:
create_box("box", [0.5, 0.4, 0.3], [0, 1, 0], 0.8)

In [ ]:
obj = g.Box(lengths=[0.5, 0.4, 0.3])
material = g.MeshLambertMaterial(color=0xff0000, opacity=0.2)
box.set_object(obj, material)